<a href="https://colab.research.google.com/github/anjiltimsina/bert_repo/blob/main/Question_Answering_model_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer , AutoModelForQuestionAnswering , TrainingArguments , Trainer , DefaultDataCollator
import torch
from google.colab import drive

In [2]:
data_collator = DefaultDataCollator()

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
from datasets import load_dataset
squad = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
train_subset = squad["train"].shuffle(seed=42).select(range(1000))
test_subset = squad["validation"].shuffle(seed=42).select(range(200))

In [ ]:
squad["train"]["question"]

In [ ]:
squad["train"]["answers"]

In [ ]:
squad["train"]["context"]

In [ ]:
squad['train']["id"]

In [ ]:
squad

In [ ]:
example = squad["train"][9]
for key in example :
  print(key , ":" , example[key])

In [ ]:
#kehi xaina yesko feature just starting_index ra ending_index thapni matrai ho tyo ditionary ma
def prepare_train_features(examples):
  tokenized_examples =tokenizer(
      examples["question"],
      examples["context"],
      truncation = "only_second", #truncate context , not question
      max_length = 384 ,
      stride =128 ,
#if context is greater than the 384 tokens then the context is split into multiple chunks , the chunks overlap by 128 tokens
      return_overflowing_tokens = True,
      return_offsets_mapping = True,
      padding="max_length"
       )

      # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    # chunks haru lai original example ma maoping garni kam garxa
    #so prediction ko time ma chai original example mai prediction bhaos bhanera
  sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

  #offset mapping le chai every token ko mapping garxa based on original context
  offset_mapping = tokenized_examples.pop("offset_mapping")

  #now labeling the examples
  tokenized_examples["start_positions"] = []
  tokenized_examples["end_positions"] = []

  #jati ota chunk ma divide bhaxa teti nai choti loop ghumxa
  for i , offsets in enumerate(offset_mapping):
    #we will label impossible answers with the index of the cls token
    #identifing the cls Token
    input_ids = tokenized_examples["input_ids"][i]
    cls_index = input_ids.index(tokenizer.cls_token_id)

#identify context and question parts
# 0 for the context , 1 For the Question and the None for the special tokens and the padding
    sequence_ids = tokenized_examples.sequence_ids(i)

#yesle chai same example bata baneko chunks haru lai eutai answer assign garna help garxa
    sample_index = sample_mapping[i]
    answers = examples["answers"][sample_index]
#if no answer is given , set the cls_index as answer
#But hamro dataset ma chai sab ko answer ta xa
#tara yo process chai maily tesko lagi jun chai chunk ma divide garenxa ni data
    if(len(answers["answer_start"]))==0:
#setting the start and end positions to the index of the cls
      tokenized_examples["start_positions"].append(cls_index)
      tokenized_examples["end_positions"].append(cls_index)
    else:
      #start and the end character index of the answer in the text
      start_char = answers["answer_start"][0]
      #total output word ma chai kati ota character xa bhanera count garxa yo code le
      end_char = start_char + len(answers["text"][0])

      #aba ko kam chai jaba samma context bhetidaina sequence_ids ma taba samma badaudai jane
      token_start_index = 0
      while sequence_ids[token_start_index] != 1:
        token_start_index +=1
      #loop bata bahira aayersi context bhetiyo

      #aba total context ko length chaiyo ni ta last ma padding gareko None chai falnu paryo ni ta
      token_end_index = len(input_ids) -1
      while sequence_ids[token_end_index] != 1:
        token_end_index -=1

  #yo two ota consective loop chalaye paxi hamilai context ko total position tha huni bho sequence_ids  ma


  #yo tala ko if condition le check garxa ki tyo chai offsets ma parxa ki nai bhanera natra
  # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
  #answer chai tyo tukrida kheri chunk ma different chunk ma huna sakxa so yo condition chai check garnu parxa
      if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1]>= end_char):
        tokenized_examples["start_positions"].append(cls_index)
        tokenized_examples["end_positions"].append(cls_index)
      else:
  #aba chai tei chunk ma result parxa bhane hamle kam garda bhayo
  #aba hamro kam chai token_start_index  and token_end_index to the end of the answers
        while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
          token_start_index +=1
        tokenized_examples["start_positions"].append(token_start_index-1)
        while offsets[token_end_index][1] >= end_char:
          token_end_index -= 1
        tokenized_examples["end_positions"].append(token_end_index + 1)

  return tokenized_examples


In [ ]:
tokenized_datasets = squad.map(prepare_train_features, batched=True, remove_columns=squad["train"].column_names)



Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    f"finetune-Bert-squad",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5 ,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size= 8,
    num_train_epochs= 1,
    weight_decay = 0.01

)


In [ ]:
#model ma tyakkai dictionary kai form ma pathau nu parxa natra aarko tarika garna gayo bhane
#gadbad dherai huna janxa hai
trainer = Trainer(
    model = model ,
    args = args,
    train_dataset = tokenized_datasets["train"].select(range(1000)) ,
    eval_dataset = tokenized_datasets["validation"].select(range(100)),
    data_collator = data_collator,
    tokenizer = tokenizer
)
#Here we can also use the metrics and save the result of weights and biases after every epoch or
#after given batch size

<ipython-input-62-4b2fea44165b>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.270890


TrainOutput(global_step=125, training_loss=4.32276708984375, metrics={'train_runtime': 4218.6964, 'train_samples_per_second': 0.237, 'train_steps_per_second': 0.03, 'total_flos': 195972567552000.0, 'train_loss': 4.32276708984375, 'epoch': 1.0})

In [ ]:
squad["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [106]:
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one

    # Tokenize the data
    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Apply the BERT model
    with torch.no_grad():  # No need to calculate gradients
        output = model(**inputs)

    # Get the predicted answer
    start_idx = torch.argmax(output.start_logits)
    end_idx = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_idx:end_idx + 1]))

    return predicted_answer.lower() == given_answer.lower()

In [ ]:
#Dheari lamo text xa bhane longformer use garda hunxa

In [107]:
from tqdm import tqdm

In [108]:
correct_count = 0
total_count = 100

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(squad['train'][i], device)

100%|██████████| 100/100 [00:02<00:00, 40.68it/s]


In [109]:

# Calculate and output the accuracy
accuracy = correct_count / total_count
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 0.00%
